In [ ]:
import os, cv2, zipfile, requests, shutil, random

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from pathlib import Path
from collections import Counter

# download and extract the dataset
DATA_DIR = Path("../data/raw")
DATA_DIR.mkdir(parents=True, exist_ok=True)

url = "https://www.kaggle.com/api/v1/datasets/download/nandanp6/cataract-image-dataset"
local_zip = DATA_DIR / "archive.zip"

if not local_zip.exists():
    print("Downloading dataset...")
    r = requests.get(url, stream=True)
    with open(local_zip, 'wb') as f:
        shutil.copyfileobj(r.raw, f)

    with zipfile.ZipFile(local_zip, 'r') as zip_ref:
        zip_ref.extractall(DATA_DIR)